In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import yaml

# !mkdir "/kaggle/working/labels"
# !cp -r "/kaggle/input/car-plate-detection/images" "/kaggle/working/images"

# Dataset

In [2]:
imgs = os.listdir('/kaggle/working/images')
imgs_train, imgs_val = train_test_split(imgs, test_size=0.05)

df = pd.read_csv('../input/car-plate-get-annotation-info-from-xml/annotation.csv')
df.head()

In [11]:
# размеры обучающей/тестовой выборок
len(imgs_train), len(imgs_val)

(411, 22)

In [24]:
cwd = '/kaggle/working/'
data = dict(
    train =  cwd + 'train.txt',
    val   =  cwd + 'val.txt',
    lr0 = 0.1,
    nc    = 1,
    names = ['licence'],
)
# .yaml файл с параметрами сети
with open(cwd + 'bgr.yaml', 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)
    
# анотации для трейна
with open(cwd + 'train.txt', 'w') as f:
    for path in imgs_train:
        f.write(cwd + 'images/' + path + '\n')
# анотации для теста
with open(cwd + 'val.txt', 'w') as f:
    for path in imgs_val:
        f.write(cwd + 'images/' + path + '\n')

In [4]:
for file in imgs:
    file = file.split('.')[0]
    bboxs = []
    for _,row in df[df['file'] == file].iterrows():
        bbox = [str(0), str(row['Xcent']), str(row['Ycent']), str(row['boxW']), str(row['boxH'])]
        bbox = ' '.join(bbox)
        bboxs.append(bbox)
    with open(cwd + 'labels/' + file + '.txt', 'w') as f:
        bboxs = '\n'.join(bboxs)
        f.write(bboxs)

# Model for Object Detection (License Detection) – YOLOv5

In [35]:
# клонируем репозиторий с yolov5
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -qr requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 14570, done.
remote: Counting objects: 100% (140/140), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 14570 (delta 86), reused 112 (delta 67), pack-reused 14430
Receiving objects: 100% (14570/14570), 13.44 MiB | 22.79 MiB/s, done.
Resolving deltas: 100% (10064/10064), done.
/kaggle/working/yolov5/yolov5
Note: you may need to restart the kernel to use updated packages.


# Train

In [8]:
!python train.py --img 1280\
--batch 8\
--epochs 50\
--optimizer SGD\
--data /kaggle/working/bgr.yaml\
--weights yolov5m6.pt\
--name result_1280_8_50_SGD

wandb: Currently logged in as: anony-mouse-429839. Use `wandb login --relogin` to force relogin
train: weights=yolov5m6.pt, cfg=, data=/kaggle/working/bgr.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=8, imgsz=1280, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=result_1280_8_50_SGD, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.2-216-g6e544d5 Python-3.7.12 torch-1.9.1 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr

In [ ]:
!python train.py --img 1280\
--batch 8\
--epochs 100\
--optimizer SGD\
--data /kaggle/working/bgr.yaml\
--weights yolov5m6.pt\
--name result_1280_8_50_SGD_01

In [32]:
!python train.py --img 640\
--batch 8\
--epochs 50\
--optimizer Adam\
--data /kaggle/working/bgr.yaml\
--weights yolov5m6.pt\
--name result_640_8_50_Adam

wandb: Currently logged in as: anony-mouse-429839. Use `wandb login --relogin` to force relogin
train: weights=yolov5m6.pt, cfg=, data=/kaggle/working/bgr.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=8, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=Adam, sync_bn=False, workers=8, project=runs/train, name=result_640_8_50_Adam, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.2-216-g6e544d5 Python-3.7.12 torch-1.9.1 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr

In [ ]:
!python train.py --img 640\
--batch 8\
--epochs 50\
--optimizer SGD\
--data /kaggle/working/bgr.yaml\
--weights yolov5m6.pt\
--name result_640_8_50_SGD

In [33]:
!python train.py --img 640\
--batch 8\
--epochs 50\
--optimizer AdamW\
--data /kaggle/working/bgr.yaml\
--weights yolov5m6.pt\
--name result_640_8_50_AdamW

wandb: Currently logged in as: anony-mouse-429839. Use `wandb login --relogin` to force relogin
train: weights=yolov5m6.pt, cfg=, data=/kaggle/working/bgr.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=8, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=AdamW, sync_bn=False, workers=8, project=runs/train, name=result_640_8_50_AdamW, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 16 (delta 9), reused 4 (delta 3), pack-reused 0
Unpacking objects: 100% (16/16), 10.31 KiB | 1.29 MiB/s, done.
From https://github.com/u